# Recipe: Get schedules for a whole year

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import datetime as dt

import pandas as pd

from cro.schedule import Client, Schedule

In [8]:
YEAR = 2022
month_dates = [dt.date(YEAR, month, 1) for month in range(1, 3)]
data: dict[Schedule, pd.DataFrame] = {}

client = Client()  # Set the station id (sid) later.

In [10]:
for sid in ("plus", "radiozurnal"):
    client.station = sid
    for date in month_dates:
        schedules = client.get_month_schedule(date)
        for schedule in schedules:
            print(schedule.date)
            data[schedule] = schedule.to_table()

In [ ]:
# Write single dataset to Excel.
for schedule, table in data.items():

    week_number = f"{schedule.date.isocalendar()[1]:02d}"
    week_start = schedule.date - dt.timedelta(days=schedule.date.weekday())  # Monday
    week_end = week_start + dt.timedelta(days=6)  # Sunday

    with pd.ExcelWriter(
        f"../data/sheet/{year}/Schedule_{schedule.station.name}_{year}W{week_number}_{week_start}_{week_end}.xlsx"
    ) as writer:
        table.to_excel(writer, index=False)

# Write concatenated datasets to Excel.
# with pd.ExcelWriter(f"../data/sheet/Schedule_{year}.xlsx") as writer:
#     pd.concat(data).to_excel(writer, index=False)

In [ ]:
# # Write single dataset to Excel.
# for _ in data:
#     with pd.ExcelWriter(f"../data/sheet/Schedule_{schedule.station.name}_M{schedule.date}.xlsx") as writer:
#         _.to_excel(writer)

# Write concatenated datasets to Excel.
with pd.ExcelWriter(f"../data/sheet/Schedule_Y{year}.xlsx") as writer:
    pd.concat(data.values()).to_excel(writer)